## Full Example #4 - Multiplayer Game of Life Example with Websockets

Let's see how we can implement a collaborative website using Websockets in FastHTML. To showcase this, we will use the famous [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway's_Game_of_Life), which is a game that takes place in a grid world. Each cell in the grid can be either alive or dead. The cell's state is initially given by a user before the game is started and then evolves through the iteration of the grid world once the clock starts. Whether a cell's state will change from the previous state depends on simple rules based on its neighboring cells' states. Here is the standard Game of Life logic implemented in Python courtesy of ChatGPT:

```python
grid = [[0 for _ in range(20)] for _ in range(20)]}
def update_grid(grid: list[list[int]]) -> list[list[int]]:
    new_grid = [[0 for _ in range(20)] for _ in range(20)]
    def count_neighbors(x, y):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        count = 0
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]): count += grid[nx][ny]
        return count
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            neighbors = count_neighbors(i, j)
            if grid[i][j] == 1:
                if neighbors < 2 or neighbors > 3: new_grid[i][j] = 0
                else: new_grid[i][j] = 1
            elif neighbors == 3: new_grid[i][j] = 1
    return new_grid
```

This would be a very pooring game if ran since the initial state of everything would stay dead. Therefore, we need a way of letting the user give an initial state before starting the game. FastHTML to the rescue!

```python
def Grid():
    cells = []
    for y, row in enumerate(game_state['grid']):
        for x, cell in enumerate(row):
            cell_class = 'alive' if cell else 'dead'
            cell = Div(cls=f'cell {cell_class}', hx_put='/update', hx_vals={'x': x, 'y': y}, hx_swap='none', hx_target='#gol', hx_trigger='click')
            cells.append(cell)
    return Div(*cells, id='grid')

@rt('/update')
async def put(x: int, y: int):
    grid[y][x] = 1 if grid[y][x] == 0 else 0
```

Above is a component for representing the game's state that the user can interact with and update on the server using cool HTMX features such as `hx_vals` for determining which cell was clicked to make it dead or alive. Now, you probably noticed that the HTTP request in this case is a PUT request, which does not return anything and this means our client's view of the grid world and the server's game state will immediately become out of sync :(. We could of course just return a new Grid component with the updated state, but that would only work for a single client, if we had more, they quickly get out of sync with each other and the server. Now Websockets to the rescue!

Websockets are a way for the server to keep a persistent connection with clients and send data to the client without explictly being requested for information, which is not possible with HTTP. Luckily FastHTML and HTMX work well with Websockets. Simply state you wish to use websockets for your app and define a websocket route:

```python
...
app = FastHTML(hdrs=(picolink, gridlink, css, htmx_ws), ws_hdr=True)

player_queue = []
async def update_players():
    for i, player in enumerate(player_queue):
        try: await player(Grid())
        except: player_queue.pop(i)
async def on_connect(send): player_queue.append(send)
async def on_disconnect(send): await update_players()

@app.ws('/gol', conn=on_connect, disconn=on_disconnect)
async def ws(msg:str, send): pass

def Home(): return Title('Game of Life'), Main(gol, Div(Grid(), id='gol', cls='row center-xs'), hx_ext="ws", ws_connect="/gol")

@rt('/update')
async def put(x: int, y: int):
    grid[y][x] = 1 if grid[y][x] == 0 else 0
    await update_players()
...
```

Here we simply keep track of all the players that have connected or disconnected to our site and when an update ocurrs, we send updates to all the players still connected via websockets. Via HTMX, you are still simply exchanging HTML from the server to the client and will swap in the content based on how you setup your `hx_swap` attribute. There is only one difference, that being all swaps are OOB. You can find more information on the HTMX websocket extension documentation page [here](https://github.com/bigskysoftware/htmx-extensions/blob/main/src/ws/README.md). You can find a full fledge hosted example of this app [here](https://game-of-life-production-ed7f.up.railway.app/).

## XT objects and HTML

These XT objects create an XML tag structure [tag,children,attrs] for `toxml()`. When we call `Div(...)`, the elements we pass in are the children. Attributes are passed in as keywords. `class` and `for` are special words in python, so we use `cls`, `klass` or `_class` instead of `class` and `fr` or `_for` instead of `for`. Note these objects are just 3-element lists - you can create custom ones too as long as they're also 3-element lists. Alternately, leaf nodes can be strings instead (which is why you can do `Div('some text')`). If you pass something that isn't a 3-element list or a string, it will be converted to a string using str()... unless (our final trick) you define a `__xt__` method that will run before str(), so you can render things a custom way.

For example, here's one way we could make a custom class that can be rendered into HTML:

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __xt__(self):
        return ['div', [f'{self.name} is {self.age} years old.'], {}]

p = Person('Jonathan', 28)
print(to_xml(Div(p, "more text", cls="container")))

<div class="container">
  <div>Jonathan is 28 years old.</div>
more text
</div>



In the examples, you'll see we often patch in `__xt__` methods to existing classes to control how they're rendered. For example, if Person didn't have a `__xt__` method or we wanted to override it, we could add a new one like this:

In [ ]:
from fastcore.all import patch

@patch
def __xt__(self:Person):
    return Div("Person info:", Ul(Li("Name:",self.name), Li("Age:", self.age)))

show(p)

Some tags from fastcore.xml are overwritten by fasthtml.core and a few are furter extended by fasthtml.xtend using this method. Over time, we hope to see others developing custom components too, giving us a larger and larger ecosystem of reusable components.

## Custom Scripts and Styling

There are many popular JavaScript and CSS libraries that can be used via a simple `Script` or `Style` tag. But in some cases you will need to write more custom code. FastHTML's [js.py](https://github.com/AnswerDotAI/fasthtml/blob/main/fasthtml/js.py) contains a few examples that may be useful as reference.

For example, to use the [marked.js](https://marked.js.org/) library to render markdown in a div, including in components added after the page has loaded via htmx, we do something like this:

```javascript
import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
import { proc_htmx} from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('%s', e => e.innerHTML = marked.parse(e.textContent));
```

`proc_htmx` is a shortcut we wrote that we wrote to apply a function to elements matching a selector, including the element that triggered the event. Here's the code for reference:

```javascript
export function proc_htmx(sel, func) {
  htmx.onLoad(elt => {
    const elements = htmx.findAll(elt, sel);
    if (elt.matches(sel)) elements.unshift(elt)
    elements.forEach(func);
  });
}
```

The [AI Pictionary example](https://github.com/AnswerDotAI/fasthtml-example/tree/main/ai_pictionary) uses a larger chunk of custom JavaScript to handle the drawing canvas. It's a good example of the type of application where running code on the client side makes the most sense, but still shows how you can integrate it with FastHTML on the server side to add functionality (like the AI responses) easily.

Adding styling with custom CSS and libraries such as tailwind is done the same way we add custom JavaScript. The [doodle example](https://github.com/AnswerDotAI/fasthtml-example/tree/main/doodle) uses [Doodle.CSS](https://github.com/chr15m/DoodleCSS) to style the page in a quirky way.


## Deploying Your App


You can deply FastHTML almost anywhere you can deploy python apps. We've tested  Railway, Replit, [HuggingFace](https://github.com/AnswerDotAI/fasthtml-hf), and [PythonAnywhere](https://github.com/AnswerDotAI/fasthtml-example/blob/main/deploying-to-pythonanywhere.md).

### Railway

[Install the Railway CLI](https://docs.railway.app/guides/cli) and sign up for an account. Set up a folder with your app as `main.py`, `requirements.txt` and `railway.toml` (copy one of the [examples](https://github.com/AnswerDotAI/fasthtml-example/tree/main/todos1) for a template). In the folder, run `railway login`, then `railway link` (to link the folder to an existing Railway project) or `railway init` to create a new one, then `railway up` to deploy. You'll see the logs as the service is built and run. 

Run `railway domain` (or click "Add a domain" in the Railway project UI) to get a URL to your app. 

Run `railway volume add -m '/app/data` to mount a folder on the cloud to your app's root folder. This is needed if we want our data to persist across restarts. The app is run in `/app` by default, so from our app anything we store in `/data` will persist across restarts.

You can add secrets like API keys that can be accessed as environment variables from your apps via ['Variables'](https://docs.railway.app/guides/variables). For example, for the image app (TODO link), you can add a REPLICATE_API_KEY variable, and then in main.py you can access it as `os.environ['REPLICATE_API_KEY']`.

We have a [deployment script](https://github.com/AnswerDotAI/fasthtml-example/blob/main/deploy.sh) that automates all of this, making it easy to deploy a FastHTML app to Railway quickly.

### Replit

Fork https://replit.com/@johnowhitaker/FastHTML-Example for a minimal example you can edit to your heart's content. `.replit` has been edited to add the right run command (`run = ["uvicorn", "main:app", "--reload"]`) and to set up the ports correctly. FastHTML was installed with `poetry add python-fasthtml`, you can add additional packages as needed in the same way. Running the app in Replit will show you a webview, but you may need to open in a new tab for all features (such as cookies) to work. When you're ready, you can deploy your app by clicking the 'Deploy' button. You pay for usage - for an app that is mostly idle the cost is usually a few cents per month.

You can store secrets like API keys via the 'Secrets' tab in the Replit project settings.

### HuggingFace

Follow the instructions in [this repository](https://github.com/AnswerDotAI/fasthtml-hf) to deploy to HuggingFace spaces.

## Where Next?

We've covered a lot of ground here! Hopefully this has given you plenty to work with in building your own FastHTML apps. If you have any questions, feel free to ask in the #fasthtml Discord channel (in the fastai community Discord). You can look through the other examples in the [fasthtml-example repository](https://github.com/AnswerDotAI/fasthtml-example) for more ideas, and keep an eye on Jeremy's [YouTube channel](https://www.youtube.com/@howardjeremyp) where we'll be releasing a number of "dev chats" related to FastHTML in the near future.